# Model Setup
Model 1 takes in img outputs moving probability
Model 2 takes in a sequence of 16 moving probs outputs falling probability

In [7]:
import os

import numpy as np
import datetime
import matplotlib.pyplot as plt

import tensorflow as tf
assert tf.__version__.startswith('2')
import tensorflow_datasets as tfds

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers


import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, RandomFlip, RandomRotation
from tensorflow.keras.utils import Sequence

 

GPU limitation

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2023-08-14 16:59:40.142685: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-14 16:59:40.158697: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-14 16:59:40.158856: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-14 16:59:40.159549: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# Building the Models

## Optimized Model 1

In [8]:
#cut a video into different frames
def cut_video(filename):
    list_of_frames = []
    cap = cv2.VideoCapture(filename)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count  = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        list_of_frames.append(frame)
   
    cap.release()
    return np.array(list_of_frames)


# loop through each frame in the array of images, take two frames at a time, 
# # loop through each pixel in the 2 frames, find the differences in values 
# in grey scale for the corresponding pixels, append the diff to the rgb array 
# representing each pixel =>>>>>>>> RGBX. Notei if first frame append 0 to rgb array
def load_images_from_folders(data_path,total): 
    images = np.zeros((total,224,224,4),dtype=float)
    counter = 0

    #loop through each vid in the folder 
    for index in range(len(data_path)): #looping through the entire folder array
        
        #variable to track whether it is the first image/frame in the video
        first_image = True

        #cut the video into different frames and save them all into an array
        array_images = cut_video(data_path[index]) # array_images[0][0][0] = single pixel

        #array to store modified pixel of each image, later on append it to images[]
        
  
        for i in range(len(array_images)-1):
            if first_image == True:
                #process for 0th and 1st frame
                images[counter] = first_frame_pixel_modification(array_images[i])
                first_image = False
                counter+=1
            
            images[counter] = two_frame_pixel_modification(array_images[i],array_images[i+1])
            counter+=1
        
    #         print("finished frame " + str(i)+ " of video " + str(index))
        
    #     print(str(counter) +" matches " + str(len(array_images)))
    #     print("finished video " + str(index))
        
    #     #after finish loop through a frame, append the frame into the images[] array
    # print("final count: " + str(counter) + "; expected count = " + str(total))
    print(counter)
    # print(images.shape)
    return images

def two_frame_pixel_modification(frame1, frame2): 
    row_frame = np.zeros((224,224,4),dtype=float)
    np.array(row_frame) 
    for r in range(len(frame1)):
        column_frame = np.zeros((224,4),dtype=float)
        for c in range(len(frame1[0])):
            #calcualte each pixel's difference 
            grey1 =  find_greyscale(frame1[r][c][2],frame1[r][c][1],frame1[r][c][0])
            grey2 =  find_greyscale(frame2[r][c][2],frame2[r][c][1],frame2[r][c][0])
            diff_grey = abs(grey1-grey2)
            new_rgbx = [frame2[r][c][2],frame2[r][c][1],frame2[r][c][0],diff_grey] #add greyscale diff to the end of the pixel array, and append it to the individual frame array
            new_rgbx = np.array(new_rgbx)
            column_frame[c] = np.array(new_rgbx)
        
        row_frame[r] = column_frame
 
        
    return row_frame 

def first_frame_pixel_modification(frame0):
    row_frame = np.zeros((224,224,4),dtype=float)
    np.array(row_frame)
    for r in range(len(frame0)):

        column_frame = np.zeros((224,4),dtype=float)
        for c in range(len(frame0[0])): 

            new_rgbx = [frame0[r][c][2],frame0[r][c][1],frame0[r][c][0],0]
            column_frame[c] = np.array(new_rgbx)

        row_frame[r] = column_frame

    return row_frame
    

def find_greyscale(r,g,b):
    grey = 0.299*r + 0.587*g + 0.114*b
    return grey

def cal_totalframes(array_of_vids):
    total_frames = 0
    for i in range(len(array_of_vids)):
        list_of_cut_frames = cut_video(array_of_vids[i])
        total_frames+= len(list_of_cut_frames)
    return total_frames






moving_path = [
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall2.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall3.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall4.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall1.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall5.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall6.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall7.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall8.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall9.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall10.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall11.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall12.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall13.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall14.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall15.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall16.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall17.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall18.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall19.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall20.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall21.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall22.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall23.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall24.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall25.mp4"
                 ]

still_path = [
                 "./../datasets/vids/splitted/new_still/resized_logitech-default1.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default2.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default3.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default4.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default5.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default6.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default7.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default8.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default9.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default10.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default11.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default12.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default13.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default14.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default15.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default16.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default17.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default18.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default19.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default20.mp4"
                 ]

moving_data = load_images_from_folders(moving_path,cal_totalframes(moving_path))
still_data = load_images_from_folders(still_path,cal_totalframes(still_path))




864
1127


### Prepare data

In [9]:
#concatenate still and movind data to create inputs and outputs
inputs1 = np.concatenate((still_data, moving_data))

outputs1 = np.concatenate((np.zeros(len(still_data)),np.ones(len(moving_data))))


#shuffle the data to prepare for training 
shuffled_indices = np.random.permutation(len(inputs1))
inputs1, outputs1 = inputs1[shuffled_indices],outputs1[shuffled_indices]

#inputs = np.concatenate((inputs,inputs))
#outputs = np.concatenate((outputs,outputs))

# inputs = inputs.astype(np.float32)
# outputs = outputs.astype(np.float32)
inputs1 = inputs1 / 255.0
outputs1 = outputs1.astype(np.float32)

#create a tensorflow dataset
with tf.device('/cpu:0'):
    dataset = tf.data.Dataset.from_tensor_slices((inputs1,outputs1))
    dataset = dataset.batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

print(inputs1.shape)
print(outputs1.shape)
print(len(dataset))

#determine train and test 
train_data = dataset.take(int(1.0*len(dataset)))

(1991, 224, 224, 4)
(1991,)
63


2023-08-14 17:14:16.808995: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3196813312 exceeds 10% of free system memory.


### Define Model

In [12]:
# input_shape = (2160,224, 224, 4)
# input_shape = (1, 224, 224, 4)
input_shape = (None, 224, 224, 4)

#data augmentation 
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomContrast(factor=0.1),
    layers.RandomZoom(0.1),
    layers.RandomTranslation(0.2, 0.2)


])


model = tf.keras.Sequential([

    # Add Convolutional and Pooling layers
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Flatten the output of the Convolutional layers before passing to Dense layers
    layers.GlobalAveragePooling2D(),
    # Flatten(),
    
    # Add Dense layers for classification
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid'),  # Using sigmoid activation for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Print the model summary to verify the input shape
model.build(input_shape)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 224, 224, 4)       0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 32)      1184      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)      

### Train Model

In [13]:
checkpoint_filepath = 'tmp/checkpoints'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


#model.fit(train_data, epochs=10, validation_data=val_data, callbacks=[model_checkpoint_callback])
model.fit(train_data, epochs=10, validation_data=tdataset, callbacks=[model_checkpoint_callback])
#model.fit(train_data, epochs=5)

model.load_weights(checkpoint_filepath)


2023-08-14 17:21:16.036437: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3196813312 exceeds 10% of free system memory.
2023-08-14 17:21:16.774970: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3196813312 exceeds 10% of free system memory.


Epoch 1/10


2023-08-14 17:21:18.760216: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8600
2023-08-14 17:21:19.148196: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


63/63 [==============================] - 7s 83ms/step - loss: 0.5603 - accuracy: 0.7268 - val_loss: 0.6871 - val_accuracy: 0.6394


2023-08-14 17:21:24.735039: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3196813312 exceeds 10% of free system memory.


Epoch 2/10
63/63 [==============================] - 5s 77ms/step - loss: 0.4656 - accuracy: 0.7951 - val_loss: 0.6455 - val_accuracy: 0.6897


2023-08-14 17:21:30.275886: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3196813312 exceeds 10% of free system memory.


Epoch 3/10
63/63 [==============================] - 5s 78ms/step - loss: 0.4505 - accuracy: 0.8112 - val_loss: 0.8661 - val_accuracy: 0.6049
Epoch 4/10
63/63 [==============================] - 5s 77ms/step - loss: 0.4364 - accuracy: 0.8172 - val_loss: 0.8043 - val_accuracy: 0.6502
Epoch 5/10
63/63 [==============================] - 5s 79ms/step - loss: 0.3958 - accuracy: 0.8393 - val_loss: 0.5069 - val_accuracy: 0.7576
Epoch 6/10
63/63 [==============================] - 5s 76ms/step - loss: 0.3448 - accuracy: 0.8679 - val_loss: 0.4069 - val_accuracy: 0.8187
Epoch 7/10
63/63 [==============================] - 5s 76ms/step - loss: 0.3601 - accuracy: 0.8629 - val_loss: 0.4265 - val_accuracy: 0.7931
Epoch 8/10
63/63 [==============================] - 5s 76ms/step - loss: 0.3409 - accuracy: 0.8709 - val_loss: 0.4847 - val_accuracy: 0.7862
Epoch 9/10
63/63 [==============================] - 5s 77ms/step - loss: 0.3328 - accuracy: 0.8724 - val_loss: 0.4177 - val_accuracy: 0.8049
Epoch 10/10
6

In [36]:
pos = 0
neg = 0
for x, y in train_data.unbatch():
    #print(x[0][0],y)
    if y > 0.5:
        pos += 1
    else:
        neg += 1

print(pos, neg)


print(train_data)

864 1127
<TakeDataset element_spec=(TensorSpec(shape=(None, 224, 224, 4), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>


### Evaluate on test dataset

In [10]:
###TESTING 
test_moving = [ "./../datasets/vids/testdata/moving/resized-test-fall1.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall2.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall3.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall4.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall5.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall6.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall7.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall8.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall9.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall10.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall11.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall12.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall13.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall14.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall15.mp4"
]

test_still = [
                "./../datasets/vids/testdata/still/resized-test-default1.mp4",
                "./../datasets/vids/testdata/still/resized-test-default2.mp4",
                "./../datasets/vids/testdata/still/resized-test-default3.mp4",
                "./../datasets/vids/testdata/still/resized-test-default4.mp4",
                "./../datasets/vids/testdata/still/resized-test-default5.mp4",
                "./../datasets/vids/testdata/still/resized-test-default6.mp4",
                "./../datasets/vids/testdata/still/resized-test-default7.mp4",
                "./../datasets/vids/testdata/still/resized-test-default8.mp4",
                "./../datasets/vids/testdata/still/resized-test-default9.mp4",
                "./../datasets/vids/testdata/still/resized-test-default10.mp4",
                "./../datasets/vids/testdata/still/resized-test-default11.mp4",
                "./../datasets/vids/testdata/still/resized-test-default12.mp4",
                "./../datasets/vids/testdata/still/resized-test-default13.mp4",
                "./../datasets/vids/testdata/still/resized-test-default14.mp4",
                "./../datasets/vids/testdata/still/resized-test-default15.mp4"]

still_tdata = load_images_from_folders(test_still,cal_totalframes(test_still))
moving_tdata = load_images_from_folders(test_moving,cal_totalframes(test_moving))




501
514


In [11]:
#concatenate still and movind data to create inputs and outputs
tinputs = np.concatenate((still_tdata, moving_tdata))
toutputs = np.concatenate((np.zeros(len(still_tdata)),np.ones(len(moving_tdata))))

# tinputs = tinputs.astype(np.float32)
# toutputs = toutputs.astype(np.float32)

tinputs = tinputs / 255.0
toutputs = toutputs.astype(np.float32)
#shuffle the data to prepare for training 
tshuffled_indices = np.random.permutation(len(tinputs))
tinputs, toutputs = tinputs[tshuffled_indices],toutputs[tshuffled_indices]

print(tinputs)
print(toutputs)

#create a tensorflow dataset
with tf.device('/cpu:0'):
    tdataset = tf.data.Dataset.from_tensor_slices((tinputs,toutputs))
    tdataset = tdataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)


# model.summary()

[[[[0.91372549 1.         0.98431373 0.06329804]
   [0.91372549 1.         0.98431373 0.06329804]
   [0.91764706 1.         0.98823529 0.06491765]
   ...
   [0.96470588 0.98431373 0.98039216 0.14509804]
   [0.96470588 0.98431373 0.98039216 0.14509804]
   [0.96470588 0.98431373 0.98039216 0.14509804]]

  [[0.91372549 1.         0.98431373 0.06329804]
   [0.91372549 1.         0.98431373 0.06329804]
   [0.91764706 1.         0.98823529 0.06491765]
   ...
   [0.96470588 0.98431373 0.98039216 0.14509804]
   [0.96470588 0.98431373 0.98039216 0.14509804]
   [0.96470588 0.98431373 0.98039216 0.14509804]]

  [[0.91372549 1.         0.98431373 0.06329804]
   [0.91372549 1.         0.98431373 0.06329804]
   [0.91764706 1.         0.98823529 0.06491765]
   ...
   [0.96470588 0.98431373 0.98039216 0.14509804]
   [0.96470588 0.98431373 0.98039216 0.14509804]
   [0.96470588 0.98431373 0.98039216 0.14509804]]

  ...

  [[0.49019608 0.50196078 0.42352941 0.30239608]
   [0.46666667 0.47843137 0.4      

In [14]:
# model.load_weights(checkpoint_filepath)
loss, accuracy = model.evaluate(tdataset)
#loss, accuracy = model.evaluate(train_data.take(100))


64/64 [==============================] - 2s 16ms/step - loss: 0.4069 - accuracy: 0.8187


### save and quantize model, turn into tflite

In [15]:
#save model1 
model.save("ktest_model1")

2023-08-14 17:22:17.125206: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ktest_model1/assets


In [16]:
def representative_dataset():
    for data, label in tf.data.Dataset.from_tensor_slices((inputs1, outputs1)).batch(1).take(100):
        yield [tf.dtypes.cast(data, tf.float32)]

with tf.device('/cpu:0'):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    tflite_quant_m1 = converter.convert()

    # Save the quantized model 1
    with open('k_m1_quant_v6.tflite', 'wb') as f: f.write(tflite_quant_m1)

INFO:tensorflow:Assets written to: /tmp/tmpqvh9ir_4/assets


/home/kevin/.local/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-08-14 17:22:20.930092: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-08-14 17:22:20.930115: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-08-14 17:22:20.930638: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpqvh9ir_4
2023-08-14 17:22:20.934909: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-08-14 17:22:20.934921: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpqvh9ir_4
2023-08-14 17:22:20.952388: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-08-14 17:22:21.007052

### Code to separate inputs of each video

In [17]:
moving_path = [
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall2.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall3.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall4.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall1.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall5.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall6.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall7.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall8.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall9.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall10.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall11.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall12.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall13.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall14.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall15.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall16.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall17.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall18.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall19.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall20.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall21.mp4"
                 ]

still_path = [
                 "./../datasets/vids/splitted/new_still/resized_logitech-default1.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default2.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default3.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default4.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default5.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default6.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default7.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default8.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default9.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default10.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default11.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default12.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default13.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default14.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default15.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default16.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default17.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default18.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default19.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default20.mp4"]


test_moving = [ "./../datasets/vids/testdata/moving/resized-test-fall1.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall2.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall3.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall4.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall5.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall6.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall7.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall8.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall9.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall10.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall11.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall12.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall13.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall14.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall15.mp4"
]

test_still = [
                "./../datasets/vids/testdata/still/resized-test-default1.mp4",
                "./../datasets/vids/testdata/still/resized-test-default2.mp4",
                "./../datasets/vids/testdata/still/resized-test-default3.mp4",
                "./../datasets/vids/testdata/still/resized-test-default4.mp4",
                "./../datasets/vids/testdata/still/resized-test-default5.mp4",
                "./../datasets/vids/testdata/still/resized-test-default6.mp4",
                "./../datasets/vids/testdata/still/resized-test-default7.mp4",
                "./../datasets/vids/testdata/still/resized-test-default8.mp4",
                "./../datasets/vids/testdata/still/resized-test-default9.mp4",
                "./../datasets/vids/testdata/still/resized-test-default10.mp4",
                "./../datasets/vids/testdata/still/resized-test-default11.mp4",
                "./../datasets/vids/testdata/still/resized-test-default12.mp4",
                "./../datasets/vids/testdata/still/resized-test-default13.mp4",
                "./../datasets/vids/testdata/still/resized-test-default14.mp4",
                "./../datasets/vids/testdata/still/resized-test-default15.mp4"]


In [18]:





def cal_total_idvFrames(vid):
    total_frame = len(cut_video(vid))
    return total_frame

def separate_aggregated_images(aggregated_images, num_images_per_video):
    separated_arrays = []
    start_idx = 0

    for num_images in num_images_per_video:
        end_idx = start_idx + num_images
        video_images = aggregated_images[start_idx:end_idx]
        separated_arrays.append(video_images)
        start_idx = end_idx

    return np.array(separated_arrays)

def separate_inputs(inputs,path):
    list_total_of_each_vid = []

    for i in range(len(path)):
        list_total_of_each_vid.append(cal_total_idvFrames(path[i]))
    
    #once obtained total of each vid, separate each
    updated_inputs = separate_aggregated_images(inputs,list_total_of_each_vid)
    return np.array(updated_inputs)


separated_moving_inputs = separate_inputs(moving_data,moving_path)
separated_still_inputs = separate_inputs(still_data,still_path)


separated_moving_test = separate_inputs(moving_tdata, test_moving)
separated_still_test = separate_inputs(still_tdata, test_still)

print(separated_moving_inputs.shape)
print(separated_still_inputs.shape)
print(separated_moving_test.shape)
print(separated_still_test.shape)

/tmp/ipykernel_774241/3015909937.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(separated_arrays)


(21,)
(20,)
(15,)
(15,)


### Use model 1 to perform prediction and generate dataset for model 2

In [19]:
with tf.device('/cpu:0'):

    def batchPredict(list_of_images_of_a_singlevid):
        probs = []

        # define intepreter for quantized modell
        intepreter = tf.lite.Interpreter(model_path="k_m1_quant_v5.tflite")
        intepreter.allocate_tensors()
        #get outputs and inputs details
        output = intepreter.get_output_details()
        input = intepreter.get_input_details()
        output_index = output[0]['index']
        input_index = input[0]['index']
        
     
        # output_index = output[0]['index']
        # input_index = input[0]['index']
        
        #set input ->invoke -> access outputs
        list_of_images_of_a_singlevid = np.array(list_of_images_of_a_singlevid, dtype=np.uint8)

        #run through model and predict the output
        for instance in list_of_images_of_a_singlevid:
            intepreter.set_tensor(input_index, np.expand_dims(instance,axis=0))
            intepreter.invoke()
            output_data = intepreter.get_tensor(output_index)
            output_data = output_data[0][0]
            # print(output_data)
            probs.append(output_data)
        
        return probs
            
        

    def batchPredictMultVids(list_of_images_of_all_vids):
        record = []
        for i in range(len(list_of_images_of_all_vids)):
            vid_probs = batchPredict(list_of_images_of_all_vids[i])
            #append sliding frames of 8 into the record
            for f in range(len(vid_probs)-7):
                record.append(vid_probs[f:f+8])
            print(len(record))
        return np.array(record)
            
    

with tf.device('/cpu:0'):
    falling_frames = []
    default_frames = []

    
    falling_frames_test = batchPredictMultVids(separated_moving_test)
    default_frames_test = batchPredictMultVids(separated_still_test)
    falling_frames = batchPredictMultVids(separated_moving_inputs)
    default_frames = batchPredictMultVids(separated_still_inputs)


   

25


KeyboardInterrupt: 

### Save output from m1 and later load in 

In [25]:


np.savetxt('model1_output_default.txt', default_frames)
np.savetxt('model1_output_fall.txt', falling_frames)
np.savetxt('model1_output_default_test.txt', default_frames_test)
np.savetxt('model1_output_fall_test.txt', falling_frames_test)

In [27]:
default_frames = np.loadtxt('model1_output_default.txt')
falling_frames = np.loadtxt('model1_output_fall.txt')
default_frames_test = np.loadtxt('model1_output_default_test.txt')
falling_frames_test = np.loadtxt('model1_output_fall_test.txt')

print(falling_frames_test)
print(default_frames_test)

#dont uncomment this
# for i in range(len(default_frames_test)):
#     print(falling_frames_test[i])


default_frames = default_frames.astype(np.uint8)
falling_frames = falling_frames.astype(np.uint8)
default_frames_test = default_frames_test.astype(np.uint8)
falling_frames_test = falling_frames_test.astype(np.uint8)

[[ 49.  49.  18. ...  42. 252. 251.]
 [ 49.  18.  49. ... 252. 251.  76.]
 [ 18.  49.  17. ... 251.  76. 255.]
 ...
 [255. 255. 255. ... 254. 254. 254.]
 [255. 255. 255. ... 254. 254. 236.]
 [255. 255. 255. ... 254. 236. 128.]]
[[29. 29. 10. ... 14. 12. 12.]
 [29. 10. 31. ... 12. 12. 17.]
 [10. 31.  9. ... 12. 17. 18.]
 ...
 [96. 31. 25. ... 29. 34. 42.]
 [31. 25. 36. ... 34. 42. 27.]
 [25. 36. 25. ... 42. 27. 86.]]


## Model 2

### Process data

In [34]:
# train data
inputs = np.concatenate((default_frames, default_frames_test,falling_frames_test,falling_frames))
outputs = np.concatenate((np.zeros(len(default_frames)),np.zeros(len(default_frames_test)),np.ones(len(falling_frames_test)), np.ones(len(falling_frames)))) #ones are falling , zeros are default
dataset_size = len(inputs)
new_indices = np.random.permutation(dataset_size) # shuffle indices to shuffle X and y at the same time
inputs, outputs = inputs[new_indices], outputs[new_indices]

train_size = int(1*dataset_size)
# train_data = tf.data.Dataset.from_tensor_slices((inputs[:train_size], outputs[:train_size])).batch(128)
X_train, y_train = inputs[:train_size], outputs[:train_size] #x = images, y = label 

print(X_train.shape,y_train.shape)

(1723, 8) (1723,)


In [35]:
#test data
inputs = np.concatenate((default_frames_test, falling_frames_test))
unshuffled_frames = inputs
outputs = np.concatenate((np.zeros(len(default_frames_test)), np.ones(len(falling_frames_test)))) #ones are falling , zeros are default
dataset_size = len(inputs)
new_indices = np.random.permutation(dataset_size) # shuffle indices to shuffle X and y at the same time
inputs, outputs = inputs[new_indices], outputs[new_indices]

train_size = int(1*dataset_size)
# train_data = tf.data.Dataset.from_tensor_slices((inputs[:train_size], outputs[:train_size])).batch(128)
X_test, y_test = inputs[:train_size], outputs[:train_size] #x = images, y = label 

print(X_test.shape,y_test.shape)


(805, 8) (805,)


### Train M2

In [41]:


model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(8,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


#Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)



In [42]:
model.fit(x=X_train, y=y_train, epochs=15,callbacks=[tensorboard_callback], validation_data=(X_test,y_test))

Epoch 1/15


54/54 [==============================] - 0s 3ms/step - loss: 15.9393 - accuracy: 0.5038 - val_loss: 2.6129 - val_accuracy: 0.6348
Epoch 2/15
54/54 [==============================] - 0s 2ms/step - loss: 1.7696 - accuracy: 0.6030 - val_loss: 1.0539 - val_accuracy: 0.6894
Epoch 3/15
54/54 [==============================] - 0s 4ms/step - loss: 0.8448 - accuracy: 0.6901 - val_loss: 0.6568 - val_accuracy: 0.7205
Epoch 4/15
54/54 [==============================] - 0s 3ms/step - loss: 0.6736 - accuracy: 0.7609 - val_loss: 0.4505 - val_accuracy: 0.8745
Epoch 5/15
54/54 [==============================] - 0s 6ms/step - loss: 0.4800 - accuracy: 0.8154 - val_loss: 0.3562 - val_accuracy: 0.8683
Epoch 6/15
54/54 [==============================] - 0s 4ms/step - loss: 0.4229 - accuracy: 0.8543 - val_loss: 0.4450 - val_accuracy: 0.8050
Epoch 7/15
54/54 [==============================] - 0s 4ms/step - loss: 0.4071 - accuracy: 0.8584 - val_loss: 0.3568 - val_accuracy: 0.9217
Epoch 8/15
54/54 [============

### Evaluate & Export

In [43]:
loss, acc = model.evaluate(X_test, y_test)
# model.summary()
model.save("model2")

26/26 [==============================] - 0s 953us/step - loss: 0.1779 - accuracy: 0.9354
INFO:tensorflow:Assets written to: model2/assets


INFO:tensorflow:Assets written to: model2/assets


In [44]:
### CONVERT TO TFLITE
converter = tf.lite.TFLiteConverter.from_keras_model(model)
model2_tflite = converter.convert()
with open('updated_kmodel2.tflite', 'wb') as f: f.write(model2_tflite)

INFO:tensorflow:Assets written to: /tmp/tmpxn46en05/assets


INFO:tensorflow:Assets written to: /tmp/tmpxn46en05/assets
2023-08-14 11:03:07.382892: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-08-14 11:03:07.382911: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-08-14 11:03:07.383051: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpxn46en05
2023-08-14 11:03:07.383692: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-08-14 11:03:07.383703: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpxn46en05
2023-08-14 11:03:07.385584: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-08-14 11:03:07.407334: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpxn46en05
2023-08-14 11:03:07.414219: I tensorflow/cc/saved_model/loader.cc:301] SavedModel

In [46]:
### QUANTIZE
def representative_dataset():
  for d in inputs:
    # d = np.expand_dims(d, axis=0)
    yield [tf.dtypes.cast(d, tf.float32)]

# print(dataset.cardinality().numpy())
# print(tf.shape(dataset))
# model.summary()

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model2 = converter.convert()
with open('k_m2_quant_v5.tflite', 'wb') as f: f.write(tflite_quant_model2)

INFO:tensorflow:Assets written to: /tmp/tmpo4l0ulw1/assets


INFO:tensorflow:Assets written to: /tmp/tmpo4l0ulw1/assets
/home/kevin/.local/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-08-14 11:03:17.997773: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-08-14 11:03:17.997792: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-08-14 11:03:17.997958: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpo4l0ulw1
2023-08-14 11:03:17.998563: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-08-14 11:03:17.998573: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpo4l0ulw1
2023-08-14 11:03:18.000368: I tensorflow/cc/saved_model/loader.cc:2

## Pass into Model 2 and get final prediction

In [47]:

# define intepreter for quantized modell
intepreter = tf.lite.Interpreter(model_path="k_m2_quant_v5.tflite")
intepreter.allocate_tensors()
#get outputs and inputs details
output = intepreter.get_output_details()
input = intepreter.get_input_details()
output_index = output[0]['index']
input_index = input[0]['index']

In [48]:
print(X_test)

[[255 255 255 ... 254 236 128]
 [133 133 150 ... 144 217 254]
 [112  96  36 ...  59 200  96]
 ...
 [ 59  52  59 ...  68  56  91]
 [239  86  96 ... 117 150  96]
 [217 255 255 ... 255 255 255]]


In [49]:
probs = []
for instance in unshuffled_frames:
            print(instance)
            intepreter.set_tensor(input_index, np.expand_dims(instance,axis=0))
            intepreter.invoke()
            output_data = intepreter.get_tensor(output_index)
            output_data = output_data[0][0]
            probs.append(output_data/255)
        
print(len(probs))

[29 29 10 31  9 14 12 12]
[29 10 31  9 14 12 12 17]
[10 31  9 14 12 12 17 18]
[31  9 14 12 12 17 18 17]
[ 9 14 12 12 17 18 17 21]
[14 12 12 17 18 17 21 36]
[12 12 17 18 17 21 36 18]
[12 17 18 17 21 36 18 14]
[17 18 17 21 36 18 14 20]
[18 17 21 36 18 14 20 72]
[17 21 36 18 14 20 72 63]
[21 36 18 14 20 72 63 36]
[36 18 14 20 72 63 36 11]
[18 14 20 72 63 36 11 39]
[14 20 72 63 36 11 39 18]
[20 72 63 36 11 39 18 11]
[72 63 36 11 39 18 11 31]
[63 36 11 39 18 11 31 12]
[36 11 39 18 11 31 12 14]
[11 39 18 11 31 12 14 20]
[39 18 11 31 12 14 20 16]
[18 11 31 12 14 20 16 14]
[11 31 12 14 20 16 14 10]
[31 12 14 20 16 14 10 10]
[12 14 20 16 14 10 10 10]
[14 20 16 14 10 10 10 16]
[45 49 45 86 86 31 42 36]
[49 45 86 86 31 42 36 45]
[45 86 86 31 42 36 45 36]
[86 86 31 42 36 45 36 39]
[86 31 42 36 45 36 39 36]
[31 42 36 45 36 39 36 34]
[ 42  36  45  36  39  36  34 204]
[ 36  45  36  39  36  34 204 144]
[ 45  36  39  36  34 204 144  56]
[ 36  39  36  34 204 144  56  27]
[ 39  36  34 204 144  56  27  27

In [50]:


default_num = default_frames_test.shape[0] #396 
falling_num = falling_frames_test.shape[0] #409

print(default_num,falling_num)

396 409


In [51]:


#testing on test data
def find_prediction_given_thresh(thresh,probs):
    prediction = []
    for i in range(len(probs)):
        if probs[i] > thresh:
            prediction.append([probs[i],True])
        else:
            prediction.append([probs[i],False])
    return prediction


def threshold_optimizer(probs):
    global_best = [-1,32434343]
    for f in range(30,95):
        threshold = f/100
        prediction = find_prediction_given_thresh(threshold,probs)
        fake_fall = 0
        fake_still = 0

        for i in range(len(prediction)):
            if i < 396:
                if prediction[i][1] == True:
                    fake_fall +=1
            else: 
                if prediction[i][1] == False:
                    fake_still +=1

        total_fakes = fake_fall + fake_still
        if total_fakes < global_best[1]:
            global_best = [f,total_fakes]
    print(global_best)
    return global_best[0]

        


final_prediction = find_prediction_given_thresh(threshold_optimizer(probs)/100,probs)
with open('test_optimized_m2_output.txt', 'w') as f:
    for i in range(len(final_prediction)):
        f.write(str(final_prediction[i][1])+"\n")
f.close()

        
    



[62, 26]
